In [6]:
import json
from pathlib import Path
#import importlib
#import parse_bullets_score as pbs
#from pathlib import Path

In [8]:

try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path.cwd()
   
bert_dir = BASE_DIR / "tmp_data"
bert_prefix = "bert_inv_ev_"
bert_suffix = "_bullet_first.json"


dst_dir =  BASE_DIR  / "../data/no_split_bullet_first"
dst_suffix = "_bullet_first_result.json"
#llama_2refute_bullet_first.json

In [10]:
def merge_bert_inv_into_result(
    bert_dir,
    dst_dir,
    bert_prefix,
    bert_suffix,
    dst_suffix
):
    models = ["llama", "mistral", "qwen"]
    modes = ["2support", "2refute", "mix"]
    prompt_types = ["basic", "internal", "claim"]

    for model in models:
        for mode in modes:
            bert_file = f"{bert_prefix}{model}_{mode}{bert_suffix}"
            bert_path = bert_dir / bert_file

            dst_file = f"{model}_{mode}{dst_suffix}"
            dst_path = dst_dir / dst_file

            #print(f"bert_path: {bert_path}")
            #print(f"dst_path: {dst_path}")

            with open(bert_path, "r", encoding="utf-8") as f:
                data_A = json.load(f)

            with open(dst_path, "r", encoding="utf-8") as f:
                data_B = json.load(f)


            bert_by_claim_id = {}

            for item in data_A:
                cid = item.get("claim_id")
                if not cid:
                    continue

                bert_by_claim_id[cid] = {}

                for ptype in prompt_types:
                    key = f"Bert_score_for_inversed_evidence_{ptype}"
                    if key in item:
                        bert_by_claim_id[cid][ptype] = item[key]["f1"]


            added = 0
            missing = []

            for item in data_B:
                cid = item.get("claim_id")

                if cid in bert_by_claim_id:
                    item["bert_inv_scores"] = bert_by_claim_id[cid]
                    added += 1
                else:
                    missing.append(cid)

            print(f"Items in A: {len(bert_by_claim_id)}")
            print(f"Items in B: {len(data_B)}")
            print(f"BERT scores added: {added}")

            if missing:
                print(f"Missing bert_inv_scores for {len(missing)} items")
                print("Example missing claim_id:", missing[:5])


            with open(dst_path, "w", encoding="utf-8") as f:
                json.dump(data_B, f, ensure_ascii=False, indent=2)

            print(f"Saved updated file: {dst_path}")


In [12]:
merge_bert_inv_into_result(bert_dir, dst_dir, bert_prefix, bert_suffix, dst_suffix)

Items in A: 245
Items in B: 245
BERT scores added: 245
Saved updated file: /Users/yingpei/Documents/kandidatDatalogi/project_Maria/code/EvidenceUtilization/inversed_evidence_Bert/../data/no_split_bullet_first/llama_2support_bullet_first_result.json
Items in A: 563
Items in B: 563
BERT scores added: 563
Saved updated file: /Users/yingpei/Documents/kandidatDatalogi/project_Maria/code/EvidenceUtilization/inversed_evidence_Bert/../data/no_split_bullet_first/llama_2refute_bullet_first_result.json
Items in A: 183
Items in B: 183
BERT scores added: 183
Saved updated file: /Users/yingpei/Documents/kandidatDatalogi/project_Maria/code/EvidenceUtilization/inversed_evidence_Bert/../data/no_split_bullet_first/llama_mix_bullet_first_result.json
Items in A: 245
Items in B: 245
BERT scores added: 245
Saved updated file: /Users/yingpei/Documents/kandidatDatalogi/project_Maria/code/EvidenceUtilization/inversed_evidence_Bert/../data/no_split_bullet_first/mistral_2support_bullet_first_result.json
Items in